In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("../data/processed/data_processed.csv")
data.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: '../data/data_processed/data_processed.csv'

In [3]:
data.shape

(3912, 4)

In [4]:
data_droped_nan = data.dropna(subset=['context_1'])

In [5]:
data_droped_nan.shape

(2368, 4)

In [6]:
data_droped_nan

,context_3,context_2,context_1,response
1,NaN,NaN,"Если не подписывать, что это учебная дз практи...",Всем добрый денек! Занимаюсь задачей QA по кор...
3,NaN,NaN,"Или не сота, но хотя бы базированное решение, ...","Retrieve and rerank sbert, думаю"
6,NaN,NaN,"Добрый день, с глупым вопросом возможно залечу...","А почему вы решили, что комната поменьше не жи..."
9,NaN,NaN,"opencv пока не пробовали, но можно.\n""Самый бо...",В вопросе была именно такая формулировка)
11,NaN,NaN,"Всем привет! \nПодскажите, пожалуйста, какие-т...",контекстов когда совсем не применим я не могу ...
...,...,...,...,...
3904,NaN,NaN,"Всем привет, подскажите пожалуйста, как запуст...","Вынести в отдельный сервис, в котлине оставить..."
3907,NaN,NaN,Это нормально когда работодатель просит чтобы ...,Только если ты уже трудоустроен)
3909,NaN,NaN,звучит как скам),Ладно
3910,NaN,звучит как скам),Ладно,дичь


In [7]:
data.dropna()

,context_3,context_2,context_1,response
40,По идее это может сделать стейбл диффужен с re...,"вот да, видел примеры с стейбл, показалось, чт...","Смотря как подойти к делу, но вряд ли стоит вр...","мне кажется, это в первую очередь вопрос, как ..."
41,"вот да, видел примеры с стейбл, показалось, чт...","Смотря как подойти к делу, но вряд ли стоит вр...","мне кажется, это в первую очередь вопрос, как ...",Контролнет работает поверх клипа с тем на что ...
42,"Смотря как подойти к делу, но вряд ли стоит вр...","мне кажется, это в первую очередь вопрос, как ...",Контролнет работает поверх клипа с тем на что ...,А можешь че-то посоветовать по этой теме почит...
43,"мне кажется, это в первую очередь вопрос, как ...",Контролнет работает поверх клипа с тем на что ...,А можешь че-то посоветовать по этой теме почит...,https://github.com/lllyasviel/ControlNet-v1-1-...
47,Всем привет! \nЗадача: классифкация на 277 кла...,В чем проблема с использованием стандартных мо...,вот кроме efficientnet в голову ничего и не пр...,"Если надо сохранять размер картинки, то можно ..."
...,...,...,...,...
3889,https://arxiv.org/pdf/1904.02689.pdf,"Пасиб за папиру, не до конца понял, как градие...",Предикт кропается по gt и по кропу считается лосс,"Да, но как оно в коде трейна реализовано?) они..."
3890,"Пасиб за папиру, не до конца понял, как градие...",Предикт кропается по gt и по кропу считается лосс,"Да, но как оно в коде трейна реализовано?) они...","не знаю что делают они, но чтобы бэкпроп не ло..."
3898,Есть три типа сегментации:\n\nсемантическая (s...,Они пошли самым простым путём: заменили прямоу...,в U-Net можно заменить слой энкодера на любой ...,"эт да, blip умеет\nиногда даже слишком\nно мне..."
3899,Они пошли самым простым путём: заменили прямоу...,в U-Net можно заменить слой энкодера на любой ...,"эт да, blip умеет\nиногда даже слишком\nно мне...",а что у BLIP'а там? Там нет эмбеддингов?\nМне ...


In [8]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

In [159]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

In [160]:
model_path = "../models/lora-5ep"

config = PeftConfig.from_pretrained(model_path)

In [161]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [162]:
model = PeftModel.from_pretrained(model, model_path)

In [163]:
input_str = "привет"

In [164]:
input_ids = tokenizer(input_str, return_tensors="pt")
generated_token_ids = model.generate(
            **input_ids,
            top_k=10,
            top_p=0.95,
            num_beams=3,
            num_return_sequences=1,
            do_sample=True,
            no_repeat_ngram_size=2,
            temperature=1.5,
            repetition_penalty=1.2,
            length_penalty=1.0,
            eos_token_id=50257,
            max_new_tokens=40,
        )
generated_output = tokenizer.decode(generated_token_ids[0], skip_special_tokens=True)
cutted_answer = generated_output
# cut off the answer if it contains the special tokens
# if "@@ПЕРВЫЙ@@" in cutted_answer:
#     cutted_answer = cutted_answer.split("@@ПЕРВЫЙ@@")[0]
# if "@@ВТОРОЙ@@" in cutted_answer:
#     cutted_answer = cutted_answer.split("@@ВТОРОЙ@@")[0]

cutted_answer

Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


'привет@@ВТОРОЙ@@Привет, как дела? 🥺@@ПЕРВЫЙ@@'

In [165]:
input_str = "где веса ламы найти?"

chat_history_ids = "@@ПЕРВЫЙ@@ " + input_str + "@@ВТОРОЙ@@"
new_input_ids = tokenizer(chat_history_ids, return_tensors='pt')
generated_token_ids = model.generate(
        **new_input_ids,
        top_k=10,
        top_p=0.95,
        num_beams=3,
        num_return_sequences=1,
        do_sample=True,
        no_repeat_ngram_size=2,
        temperature=1.4,
        repetition_penalty=1.2,
        length_penalty=1.0,
        eos_token_id=50257,
        max_new_tokens=40,
        pad_token_id=tokenizer.eos_token_id
    )

context_with_response = [tokenizer.decode(generated_token_ids[i]) for i in range(len(generated_token_ids))]
context_with_response

['@@ПЕРВЫЙ@@ где веса ламы найти?@@ВТОРОЙ@@В гугле по картинкам.@@ПЕРВЫЙ@@']

In [124]:
import torch

torch.__version__

'2.0.1'

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('tinkoff-ai/ruDialoGPT-small')
inputs = tokenizer('@@ПЕРВЫЙ@@ где найти веса ламы? @@ВТОРОЙ@@', return_tensors='pt')
generated_token_ids = model.generate(
    **inputs,
    top_k=5,
    top_p=0.9,
    num_beams=3,
    num_return_sequences=1,
    do_sample=True,
    no_repeat_ngram_size=2,
    temperature=1.7,
    repetition_penalty=1.2,
    length_penalty=1.0,
    eos_token_id=50257,
    max_new_tokens=40,
    pad_token_id=tokenizer.eos_token_id
)
context_with_response = [tokenizer.decode(sample_token_ids) for sample_token_ids in generated_token_ids]
context_with_response

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['@@ПЕРВЫЙ@@ где найти веса ламы? @@ВТОРОЙ@@В интернете. Там много чего можно найти. Например, в гугле. Или в яндекс картинках.@@ПЕРВЫЙ@@']

In [29]:
import requests
requests.__version__

'2.28.2'

In [27]:
!pip3 install telebot

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
  Using cached telebot-0.0.5-py3-none-any.whl (4.8 kB)
  Using cached pyTelegramBotAPI-4.13.0.tar.gz (232 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.13.0-py3-none-any.whl size=214546 sha256=42b335ca9dbe06e3b57c425bcf67dd685b28c9c9f6624720ac94fff15edef1c7
  Stored in directory: /Users/macbook/Library/Caches/pip/wheels/6d/05/82/5f51ca10e3939eb47f61064a76b1fb027ff69927a25a3c5190
Successfully built pyTelegramBotAPI

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLi